<a href="https://colab.research.google.com/github/krystaldowling/MAST30034_Final_Project-/blob/machine_learning_doc2vec/Edited_classifier_doc2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import scipy
import pickle

from sklearn.model_selection import train_test_split

news_data = pd.read_csv("/content/drive/My Drive/Data/final_preproccessed_data.csv")

news_data.index.name = 'index'

news_train, news_test, label_train, label_test = train_test_split(news_data.loc[:,['title','text']], 
                                                        news_data.loc[:,'label'], test_size=0.33, 
                                                        random_state=88)

In [ ]:
news_train

,title,text
index,,
20183,"Policy Shift Helps Coal, but Other Forces May ...",Many fossil fuel executives are celebrating Pr...
1750,Dr. David Duke and Adrian Salbuch : The intern...,Dr. David Duke and Adrian Salbuch : The intern...
1710,Donald Trump Jr. Met With Russian Advocates Ab...,"According to The Wall Street Journal , Preside..."
3801,Why Trump Won and Why Clinton Lost :,Why Trump Won and Why Clinton Lost\nHillary C...
24095,"The Battle of ‘Miss Saigon’: Yellowface, Art a...",“Miss Saigon” started with an audacious idea ...
...,...,...
24938,Want to Find Fulfillment at Last? Think Like a...,"STANFORD, CALIF. — Take out your flow journ..."
2481,Jonathan Haidt: the Moral Roots of Liberals an...,Jonathan Haidt: the Moral Roots of Liberals an...
4047,Trump: A people’s ‘new world order’ taking shape?,Trump: A people’s ‘new world order’ taking sha...


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import gensim
from gensim.test.utils import get_tmpfile

documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(common_texts)]
model = Doc2Vec(documents, vector_size=100, window=2, min_count=1, workers=4)

fname = get_tmpfile("my_doc2vec_model")

model.save(fname)
model = Doc2Vec.load(fname)  # you can continue training with the loaded model!
model.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)
vector = model.infer_vector(["system", "response","hello"])
print(vector)

df = news_data

def doc2vec_fun(x):
    L = []
    for elem in x.split(' '):
        elem = (elem.replace('\n', '').replace(',', '').replace('.', '').
        replace('”', '').replace('?', '').replace('\'', '').lower().
        replace('“', '').replace(']', '').replace('[', ''))
        L.append(elem)
    vector = model.infer_vector(L)
    return vector
df['doc2vec_title']=df['title'].map(lambda x:doc2vec_fun(x))
df['doc2vec_text']=df['text'].map(lambda x:doc2vec_fun(x))

print(df.head(10))

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


[-2.0719261e-04  9.3791459e-04 -2.3677859e-03 -3.6256821e-03
 -1.6408011e-03  8.2280560e-05 -3.3505557e-03  9.1662235e-04
  2.4754032e-03 -3.7782160e-03 -3.7631160e-03  2.8763455e-03
 -2.9673378e-03  3.5750235e-03 -4.2815283e-03 -3.0055540e-03
 -3.5983924e-04 -6.1778328e-04  4.3127441e-04 -4.5878636e-03
  3.9157950e-04 -2.8197879e-03 -4.9738945e-03  1.4794740e-03
  1.8967736e-03  2.6587392e-03  2.1336756e-03 -8.6815760e-04
  4.4274693e-03  3.5133823e-03  4.0685371e-03 -5.8008998e-04
 -1.9733625e-04  3.6512176e-03  1.3288347e-03 -1.2363839e-03
 -7.4724376e-04 -4.5189182e-03  4.8285048e-03 -4.1141612e-03
  3.1995533e-03 -1.1711784e-03 -2.5293510e-03  2.4885284e-03
 -2.1617934e-03  2.2717456e-04  3.4539236e-03 -3.7732692e-03
  3.7630135e-03  3.2077776e-03 -7.8347279e-04 -6.5426249e-04
 -3.0411647e-03  2.7766111e-03  4.1049998e-03  4.4236747e-03
 -5.0775555e-04  3.3567557e-03  9.0512511e-04 -3.3661879e-03
  5.5737724e-04  9.8913733e-04 -5.0508545e-05  2.5746832e-03
 -4.5339207e-04  2.83023

In [33]:
# this is only the text data which has been converted to a data frame.
# each row is one article and each column is an element of the vector. 

vectorised_data = pd.DataFrame(df['doc2vec_text'].array, columns = range(100))

# once we have vectorised the entire datset then split into train and test! 
X_train, X_test, y_train, y_test = train_test_split(vectorised_data,df['label'])

# this is only the text which has been vectorised but maybe run the classifer on just this first and then
# have a look at doing it for both title as well as the text

In [ ]:
df_test = news_test

df_test['doc2vec_title']=df_test['title'].map(lambda x:doc2vec_fun(x))
df_test['doc2vec_text']=df_test['text'].map(lambda x:doc2vec_fun(x))

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import mutual_info_classif
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.svm import SVC

def train_models(X_train, X_test, y_train, y_test):
    '''
    function performs feature selection and fits Gaussian Naive Bayes, 
    Multinomial Naive Bayes, Logistic Regression and Support Vector Classification
    on training data - finds the accuracy score of the model on the test data
    '''
  

    models = [GaussianNB(),
            MultinomialNB(),
            SVC(),
            LogisticRegression()]

    titles = ['GNB',
            'MNB',
            'SVC',
            'Logistic Regression']

    k = 10

    x2 = SelectKBest(chi2, k)
    x2.fit(X_train,y_train)
    X_train_x2 = x2.transform(X_train)
    X_test_x2 = x2.transform(X_test)


    mi = SelectKBest(mutual_info_classif, k)
    mi.fit(X_train,y_train)
    X_train_mi = mi.transform(X_train)
    X_test_mi = mi.transform(X_test)


    Xs = [(X_train_x2, X_test_x2), (X_train_mi, X_test_mi)]
     
  
    X_names = ['x2', 'mi']
    

    for title, model in zip(titles, models):
        print('\n',title, '(with k=',k,'features):')
        for X_name, X in zip(X_names, Xs):
            X_train_t, X_test_t = X
            model.fit(X_train_t.todense(), y_train)
            acc = model.score(X_test_t.todense(), y_test)
            print(X_name, '\t acc',  acc)

    return

train_models(df['doc2vec_text'], df_test['doc2vec_text'], label_train, label_test)

ValueError: ignored